In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
import urllib
import json

In [2]:
# Calculates centroid of a convex polygon from a list of vertices
def centroid(vertices):
     _x_list = [vertex [0] for vertex in vertices]
     _y_list = [vertex [1] for vertex in vertices]
     _len = len(vertices)
     _x = sum(_x_list) / _len
     _y = sum(_y_list) / _len
     return(_x, _y)

In [3]:
# Pass request to site for GeoJSON
runway_url = "https://opendata.arcgis.com/datasets/4d8fa46181aa470d809776c57a8ab1f6_0.geojson"
req = urllib.request.urlopen(runway_url)

In [4]:
# Read into JSON
data = req.read()
runways_json = json.loads(data.decode('utf-8'))

In [5]:
# Read JSON into dataframe
runways_df = pd.json_normalize(runways_json, record_path =['features'])
# Add lat/long for runway center
runways_df['Runway.Latitude'] = ''
runways_df['Runway.Longitude'] = ''

In [6]:
# Extract list of coordinates from nested list
for i in runways_df.index:
    runways_df.at[i, 'geometry.coordinates'] = runways_df['geometry.coordinates'][i][0]

In [7]:
# Iterate through dataframe to calculate center of runway and split into lat/long values
for i in runways_df.index:
    coords = centroid(runways_df['geometry.coordinates'][i])
    runways_df.at[i, 'Runway.Latitude'] = coords[0]
    runways_df.at[i, 'Runway.Longitude'] = coords[1]

In [8]:
runways_df

,type,properties.OBJECTID,properties.GLOBAL_ID,properties.AIRPORT_ID,properties.DESIGNATOR,properties.LENGTH,properties.WIDTH,properties.DIM_UOM,properties.COMP_CODE,properties.LIGHTACTV,...,properties.US_LOW,properties.US_HIGH,properties.US_AREA,properties.PACIFIC,properties.Shape__Area,properties.Shape__Length,geometry.type,geometry.coordinates,Runway.Latitude,Runway.Longitude
0,Feature,1,C747AD56-6CCF-4130-9255-643A4306CA51,656D38F0-F1FE-49A8-AB4F-677281616EF8,05/23,7790,200,FT,ASPH,2.0,...,0,0,0,0,1.889291e-05,0.064914,Polygon,"[[-176.65729554655, 51.8780991687588, 0.0], [-...",-176.645,51.8825
1,Feature,2,EA83C6B6-306D-4AD0-9BBC-3534B05CA636,F39AFCD2-D07F-4F41-96CF-08B79A271EAB,04/22,3120,50,FT,GRAVE,NaN,...,0,0,0,0,2.138160e-06,0.029114,Polygon,"[[-154.176046318343, 56.9404912496556, 0.0], [...",-154.182,56.939
2,Feature,3,6D8AC274-5B38-4994-8559-05B048032A21,C0EE48D3-E3AD-404E-945D-F404E345020D,01/19,3300,60,FT,GRAVE,2.0,...,0,0,0,0,3.042399e-06,0.023729,Polygon,"[[-161.489373070729, 60.9178863636435, 0.0], [...",-161.493,60.9146
3,Feature,4,84E294FE-4120-472E-9186-7FE999B4BA7E,26D96486-FA29-4866-93EB-2EEEB7FA7144,E/W,5000,300,FT,WATER,NaN,...,0,0,0,0,2.756513e-04,0.668321,Polygon,"[[-0.333313083653724, -0.000411438872305, 0.0]...",-0.19998,-8.06404e-05
4,Feature,5,135362D2-1BD6-45A1-86B6-0B7BA2314384,26D96486-FA29-4866-93EB-2EEEB7FA7144,NW/SE,5000,300,FT,WATER,NaN,...,0,0,0,0,2.756513e-04,0.668321,Polygon,"[[-0.333313083653724, -0.000411438872305, 0.0]...",-0.19998,-8.06404e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,Feature,24092,4F0E067C-87A7-4AFC-A820-A899EB4DCEC0,86E720F3-79ED-41F2-B8AD-7BAC41EE7F35,09/27,960,50,FT,GRASS,NaN,...,0,0,0,0,4.645310e-07,0.007036,Polygon,"[[-87.0992089862132, 39.1583417198393, 0.0], [...",-87.1006,39.1585
24092,Feature,24093,F5DDA7B5-E9F5-4432-9177-50E550AE39E2,A05B01BE-4052-4344-9EC0-937C3F78CF50,H1,45,45,FT,CONC,NaN,...,0,0,0,0,2.183606e-08,0.000601,Polygon,"[[-94.363725283114, 45.967097533604, 0.0], [-9...",-94.3638,45.967
24093,Feature,24094,4C580E1B-BB6C-422E-BCA9-45BD444E75A8,6B007B47-12DC-433A-907C-A17DF1F68704,H1,40,40,FT,CONC,NaN,...,0,0,0,0,1.504122e-08,0.000494,Polygon,"[[-93.7091468897265, 36.9694949229938, 0.0], [...",-93.7092,36.9695
24094,Feature,24095,FB917C6C-DC51-4219-AB77-90D1624CB360,AF6FE437-1EEC-437C-9F10-CFE7B81CF4E8,H1,40,40,FT,CONC,NaN,...,0,0,0,0,1.507442e-08,0.000494,Polygon,"[[-94.3221274719381, 37.1369976969745, 0.0], [...",-94.3222,37.137


In [15]:
# Export dataframe to CSV
runways_df.to_csv('runways_cleaned.csv')